# diagnostics

In [1]:
import tensorflow as tf
print(tf.__version__)

2.15.1


# Setup

This notebook mirrors `emotion_regressor.ipynb`, but swaps the GoEmotions transformer for the **Appraisal enISEAR** pre-trained *appraisal-from-text* predictor.

It produces a **stateless appraisal vector per utterance** (no derivatives / EMA here — that stays in TrendTracker).

In [2]:
# Environment / dependencies
# --------------------------
# This appraisal model repo expects an older TF/Keras stack (per its README).
# Recommended (conda) env:
#
#   conda create -n tf114 python=3.8
#   conda activate tf114
#   pip install numpy==1.16.4 pandas sklearn keras==2.3.0 gensim tensorflow==1.14
#
# If you try to run this in your "main" env, it may or may not load the .h5 successfully.
# (Keras/TF version mismatches are a classic footgun.)

import os
import sys
import subprocess
from pathlib import Path

import numpy as np
import pandas as pd


# Building the appraisalRegressor

We treat appraisals as an **orthogonal signal** to the GoEmotions 28d vector.

Under the hood, we reuse the repo’s own script (`b_appraisals_from_text.py`) in **inference mode** with its provided pre-trained weights. This avoids re-implementing their preprocessing/tokenizer/embedding logic.

In [3]:
import sys
import subprocess
from pathlib import Path
import numpy as np
import pandas as pd

APPRAISAL_REPO_URL = "https://github.com/bluzukk/appraisal-emotion-classification.git"
APPRAISAL_REPO_DIR = Path("appraisal-emotion-classification")

# enISEAR V1 appraisal dimensions
APPRAISAL_LABELS = [
    "Attention",
    "Certainty",
    "Effort",
    "Pleasant",
    "Responsibility",
    "Control",
    "Circumstance",
]

PRETRAINED_MODEL_REL = Path("pre-trained/enISEAR_V1_appraisal_predictor.h5")
SCRIPT_REL = Path("impl-main-experiments/b_appraisals_from_text.py")


def ensure_appraisal_repo() -> Path:
    """Clone the enISEAR appraisal repo if it's not present, and patch legacy Keras imports."""
    if not APPRAISAL_REPO_DIR.exists():
        print(f"Cloning: {APPRAISAL_REPO_URL}")
        subprocess.run(["git", "clone", APPRAISAL_REPO_URL], check=True)

    # --- AUTO-PATCH legacy Keras import ---
    script_path = APPRAISAL_REPO_DIR / SCRIPT_REL
    if script_path.exists():
        text = script_path.read_text()
        old = "from keras.preprocessing.sequence import pad_sequences, sequence"
        new = (
            "from keras.preprocessing.sequence import pad_sequences\n"
            "import keras.preprocessing.sequence as sequence"
        )
        if old in text:
            script_path.write_text(text.replace(old, new))
            print("Patched enISEAR script: fixed keras.preprocessing.sequence import")

    return APPRAISAL_REPO_DIR


class AppraisalRegressor:
    """
    Stateless appraisal vectorizer (enISEAR V1).

    API mirrors EmotionRegressor:
        .predict(text: str) -> np.ndarray shape (7,)
    """

    def __init__(
        self,
        repo_dir=APPRAISAL_REPO_DIR,
        dataset: str = "enISEAR_V1",
        model_rel_path=PRETRAINED_MODEL_REL,
        quiet: bool = True,
    ):
        self.repo_dir = Path(repo_dir)
        self.dataset = dataset
        self.model_path = (self.repo_dir / Path(model_rel_path)).resolve()
        self.script_path = (self.repo_dir / SCRIPT_REL).resolve()
        self.quiet = quiet

        if not self.repo_dir.exists():
            raise FileNotFoundError(
                f"Repo dir not found: {self.repo_dir}. Run ensure_appraisal_repo() first."
            )
        if not self.model_path.exists():
            raise FileNotFoundError(
                f"Pretrained model not found: {self.model_path}"
            )
        if not self.script_path.exists():
            raise FileNotFoundError(
                f"Script not found: {self.script_path}"
            )

    def predict(self, text: str) -> np.ndarray:
        """Predict 7 appraisal dimensions for a single utterance."""
        text = (text or "").strip()

        work_dir = self.script_path.parent
        tmp_in = work_dir / "TextInstances.csv"
        tmp_out = work_dir / "TextInstances_appraisals.csv"

        # Write input CSV expected by enISEAR script
        pd.DataFrame({"Sentence": [text]}).to_csv(tmp_in, index=False)

        cmd = [
            sys.executable,
            self.script_path.name,
            "-d", self.dataset,
            "--annotate", str(tmp_in.name),
            "--loadmodel", str(self.model_path),
            "--continous", # toggles from binary mode to continuous, like real regression
        ]
        
        
        if self.quiet:
            cmd.append("--quiet")

        proc = subprocess.run(
            cmd,
            cwd=work_dir,
            capture_output=True,
            text=True,
        )

        if proc.returncode != 0:
            raise RuntimeError(
                "enISEAR inference failed.\n\n"
                "STDOUT:\n"
                + (proc.stdout or "")
                + "\n\nSTDERR:\n"
                + (proc.stderr or "")
            )

        if not tmp_out.exists():
            raise FileNotFoundError(
                f"Expected output not found: {tmp_out}"
            )

        df = pd.read_csv(tmp_out)

        if all(col in df.columns for col in APPRAISAL_LABELS):
            vals = df.loc[0, APPRAISAL_LABELS].to_numpy(dtype=float)
        else:
            cols = [c for c in df.columns if c.lower() != "sentence"]
            vals = df.loc[0, cols].to_numpy(dtype=float)

        # Cleanup temp files
        for p in (tmp_in, tmp_out):
            try:
                p.unlink()
            except FileNotFoundError:
                pass

        return vals

# Demo of 7-dimensional appraisal_vector

## Hard-coded example

In [4]:
# Initialize once
_ = ensure_appraisal_repo()
appraisal_regressor = AppraisalRegressor(repo_dir=APPRAISAL_REPO_DIR)

# Test utterances
tests = [
    "LIAR!",
    "I feel trapped and I don't know what to do.",
    "I worked really hard and it still went wrong, and I'm blaming myself.",
    "This is wonderful news and I'm excited for the future."
]

# Collect results
rows = []
for t in tests:
    vals = appraisal_regressor.predict(t)
    rows.append([t] + list(vals))

# Build dataframe
appraisal_df = pd.DataFrame(
    rows,
    columns=["utterance"] + APPRAISAL_LABELS
)

display(appraisal_df)

,utterance,Attention,Certainty,Effort,Pleasant,Responsibility,Control,Circumstance
0,LIAR!,0.528353,0.539859,0.505152,0.512351,0.516323,0.500644,0.521535
1,I feel trapped and I don't know what to do.,0.093578,0.381065,0.897412,0.001396,0.904527,0.812091,0.085965
2,"I worked really hard and it still went wrong, ...",0.803091,0.084078,0.961688,0.005474,0.785140,0.047193,0.517921
3,This is wonderful news and I'm excited for the...,0.996712,0.997619,0.000183,0.950602,0.024001,0.007626,0.284756


## Live use (single-input)

In [ ]:
# user_input = input("Enter text: ").strip()
# appraisal_vector = appraisal_regressor.predict(user_input)
# print(pd.DataFrame([appraisal_vector], columns=APPRAISAL_LABELS))


# Mustafar Demo (Appraisals)

This mirrors your EmotionRegressor Mustafar demo, but outputs the appraisal vector instead.

**Note:** This is still stateless — no derivatives / EMA here.

In [5]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Absolute path, since the notebook lives elsewhere
ROTS_PATH = "/Users/dansantoro/Desktop/project_therapy/datasets/rots.csv"

# Load dataset
rots_df = pd.read_csv(ROTS_PATH)

# Filter Anakin lines, preserve order
anakin_df = rots_df[rots_df["SPEAKER"] == "Anakin"].reset_index(drop=True)

# Collect utterances + appraisal vectors
rows = []
for utterance in anakin_df["UTTERANCE"]:
    vec = appraisal_regressor.predict(utterance)
    rows.append([utterance] + list(vec))

# Build TrendTracker-style dataframe
anakin_appraisals = pd.DataFrame(
    rows,
    columns=["utterance"] + APPRAISAL_LABELS
)

display(anakin_appraisals)


,utterance,Attention,Certainty,Effort,Pleasant,Responsibility,Control,Circumstance
0,I saw your ship. What are you doing out here?,0.256593,0.963094,0.035091,0.053475,0.138297,0.093934,0.032723
1,What things?,0.277851,0.813272,0.239614,0.271415,0.457599,0.477211,0.305191
2,Obi-Wan is trying to turn you against me.,0.554759,0.474136,0.935339,0.044194,0.289271,0.209437,0.128032
3,Us?,0.435769,0.608821,0.770465,0.078764,0.329508,0.181736,0.336485
4,"Love won't save you, Padme. Only my new powers can do that.",0.824199,0.992693,0.150960,0.806334,0.708482,0.783695,0.051184
5,I won't lose you the way I lost my mother. I am becoming more powerful than any Jedi has ever dreamed of. And I'm doing it for you. To protect you.,0.404549,0.695195,0.804662,0.021452,0.905577,0.065965,0.685610
6,"Don't you see? We don't HAVE to run away anymore. I have brought PEACE to the Republic. I am more powerful than the Chancellor, I—I can overthrow him! And together, you and I can rule the galaxy, make things the way we want them to be!",0.641010,0.782953,0.887246,0.035722,0.857221,0.502211,0.351791
7,I don't want to hear any more about Obi-Wan. The Jedi turned against me—don't YOU turn against me!,0.055215,0.469591,0.501432,0.002912,0.214469,0.018463,0.262341
8,Because of Obi-Wan?,0.136375,0.885261,0.306162,0.063782,0.320950,0.198589,0.674925
9,LIAR!,0.528353,0.539859,0.505152,0.512351,0.516323,0.500644,0.521535
